# Müzik Sınıflandırması

Metin analizinin temel adımları aşağıdaki gösterilecektir. Uygulama sırasında, ön veri hazırlama aşamaları, makine öğrenmesi sınıflandırma işlemleri gerçekleştirilecektir.


Örnek çalışmada farklı türlerdeki şarkıların sözlerini içeren bir veri seti kullanılacaktır. https://www.kaggle.com/mehedihasan9021/movie-script-dataset


İlk olarak kullanılacak verinin pandas ile sisteme aktarılması gerekmektedir.

In [36]:
import pandas as pd
data = pd.read_csv("dataset.csv")
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


"Müzik Türü (genre)" ve "Şarkı Sözleri (lyrics)" sütunları Dataframeden ayrılarak "columns" adında bir dataframe oluşturulmaktadır.

In [37]:
columns = data[['genre', 'lyrics']]
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


Hangi tür müziklerin olduğunu inceleniyor.

In [38]:
columns['genre'].unique()

array(['Christian', 'Country', 'Hip-Hop', 'Pop', 'Rock', 'R&B'],
      dtype=object)

In [39]:
columns['genre'].value_counts()

Pop          100
Rock          95
Christian     94
Hip-Hop       91
R&B           91
Country       87
Name: genre, dtype: int64

Doğal dil işleminin kısa sürmesi amacıyla çalışmada sadece seçilen iki müzik türü kullanılacaktır. "Rock" ve "Christian"

In [40]:
columns = columns[(columns.genre == 'Country') | (columns.genre == 'Christian')]

## Text Verisinin Önişlemlerinin Gerçekleştirilmesi

Metin analizi ve doğal dil işleme gibi makine öğrenmesi modellerini eğitmek amacıyla metin verilerini kullanmak için bunların uygun bir formata getirilmesi gerekmektedir. Aşağıdaki ön hazırlık adımları bu konuda bize yardımcı olacaktır. 

### Küçük Harfe Dönüştürme

String değişkenlerde büyük harf küçük harf duyarlılığı olduğu için analizlerde sorun olmaması için öncelikle tüm metin ifadelerinin küçük harfe dönüştürülür.

In [41]:
lowered = columns['lyrics'].str.lower()

### Noktalama işaretlerinin kaldırılması

Tüm harfler küçük harflere dönüştürüldükten sonra yapılması gereken bir sonraki işlem noktalama işaretlerinin kaldırılmasıdır. Burada "re" kütüphanesinden yararlanılacaktır.

In [42]:
import re

reg = re.compile('[^a-zA-Z ]')
for i, line in enumerate(lowered):
    lowered[i] = reg.sub('', line)

In [43]:
columns['lowered'] = lowered
columns['lowered']

C:\Users\kubil\AppData\Local\Temp\ipykernel_8444\1032143912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['lowered'] = lowered


0       who am i that the lord of all the earth would...
1       glory revealed  by his wounds he was pierced ...
2       lord of heaven and earth lord of all creation...
3       i can only imagine what it will be like when ...
4       i am not skilled to understand what god has w...
                             ...                        
176     one evening as the sun went down and the jung...
177     i see the questions in your eyes i know whats...
178     big orange ball sinkin in the water toes in t...
179     dolly parton greatest hits jolene jolene jole...
180     please help me im falling in love with you cl...
Name: lowered, Length: 181, dtype: object

### Simgeleştirme

Mevcut metinler ayrı kelimelere bölünmelidir. Bu işleme tokenization (simgeleştirme) adı verilmektedir. Bu işlemi yapmak için "nltk" kütüphanesinden yararlanılmaktadır.

In [44]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kubil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

C:\Users\kubil\AppData\Local\Temp\ipykernel_8444\926612404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['tokened'] = tokened


### Durdurma Sözcüklerinin Kaldırılması

Durdurma sözcükleri, belirli bir anlam yükü taşımayan ve bu nedenle sonraki sınıflandırma görevinde hiçbir şekilde yardımcı olmayan yaygın sözcüklerdir. Bu tür kelimeleri gürültü olarak kabul eder ve buna göre sileriz. Farklı dillerde belirli bir durdurma sözcükleri külliyatı vardır. Artık İngilizce metinlerle uğraştığımız için, İngilizce için durdurma sözcükleri çıkaracağız:

In [46]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kubil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:  
    #a = re.sub('[!@#$]', '', a)
    without_stop.append(" ".join(a))
columns['without_stop'] = without_stop
print(columns['without_stop'])

0      lord earth would care know name would care fee...
1      glory revealed wounds pierced transgressions c...
2      lord heaven earth lord creation lord heaven ea...
3      imagine like walk side imagine eyes see face i...
4      skilled understand god willed god planned know...
                             ...                        
176    one evening sun went jungle fire burning track...
177    see questions eyes know whats weighing mind su...
178    big orange ball sinkin water toes sand couldnt...
179    dolly parton greatest hits jolene jolene jolen...
180    please help im falling love close door temptat...
Name: without_stop, Length: 181, dtype: object


C:\Users\kubil\AppData\Local\Temp\ipykernel_8444\3495880655.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['without_stop'] = without_stop


### Lemmatizasyon

Aynı kelimenin farklı biçimleri vardır. Örneğin, 'danslar', 'dans' kelimesinin bir biçimidir. Lemmatizasyon, kelimenin orijinal biçimi olan lemmaya indirgemedir. Ayrıca, kelimenin farklı biçimlerinin ayrı kelimeler olarak kabul edilmemesi için verilerimize uygulanması gerekir.

Bazı lemmatizerler farklı dillerde farklı etkilerle kendilerini gösterirler. WordNetLemmatizer, İngilizce ile çalışmak için etkilidir

In [48]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kubil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kubil\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [49]:
lemmatizer = WordNetLemmatizer()
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma
columns

C:\Users\kubil\AppData\Local\Temp\ipykernel_8444\1886906035.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['lemmatized'] = lemma


,genre,lyrics,lowered,tokened,without_stop,lemmatized
0,Christian,"Who am I, that the Lord of all the earth Woul...",who am i that the lord of all the earth would...,"[who, am, i, that, the, lord, of, all, the, ea...",lord earth would care know name would care fee...,lord earth would care know name would care fee...
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...,"[glory, revealed, by, his, wounds, he, was, pi...",glory revealed wounds pierced transgressions c...,glory revealed wounds pierced transgressions c...
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...,"[lord, of, heaven, and, earth, lord, of, all, ...",lord heaven earth lord creation lord heaven ea...,lord heaven earth lord creation lord heaven ea...
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...,"[i, can, only, imagine, what, it, will, be, li...",imagine like walk side imagine eyes see face i...,imagine like walk side imagine eyes see face i...
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...,"[i, am, not, skilled, to, understand, what, go...",skilled understand god willed god planned know...,skilled understand god willed god planned know...
...,...,...,...,...,...,...
176,Country,One evening as the sun went down And the jung...,one evening as the sun went down and the jung...,"[one, evening, as, the, sun, went, down, and, ...",one evening sun went jungle fire burning track...,one evening sun went jungle fire burning track...
177,Country,I see the questions in your eyes I know whats...,i see the questions in your eyes i know whats...,"[i, see, the, questions, in, your, eyes, i, kn...",see questions eyes know whats weighing mind su...,see questions eyes know whats weighing mind su...
178,Country,Big orange ball sinkin in the water Toes in t...,big orange ball sinkin in the water toes in t...,"[big, orange, ball, sinkin, in, the, water, to...",big orange ball sinkin water toes sand couldnt...,big orange ball sinkin water toes sand couldnt...
179,Country,"Dolly Parton Greatest Hits Jolene Jolene, jol...",dolly parton greatest hits jolene jolene jole...,"[dolly, parton, greatest, hits, jolene, jolene...",dolly parton greatest hits jolene jolene jolen...,dolly parton greatest hits jolene jolene jolen...


### Verilerin Bölünmesi 

x_train - modeli eğittiğimiz metinler. Bu durumda, tüm hazırlık aşamalarından geçmiş verileri içerdiğinden lemmatized sütunu kullanıyoruz.

y_train - modelin üzerinde eğitildiği metinlere karşılık gelen türler - tür sütunu

x_test - modelin türü tahmin etmeyi ne ölçüde öğrendiğini kontrol edeceğimiz veri kümesindeki tam olarak aynı metinler

y_test - x_test'e karşılık gelen türler. yani, öğrenme kalitesini değerlendirmek için bu değişkenin içeriğine ne kadar tahminin karşılık geldiğine bakarız.

In [50]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.6)
for x, y in zip(x_test, y_test):
    print(x + "\n\t" + y + "\n")

worthy lamb slain holy holy sing new song sits heavens mercy seat worthy lamb slain holy holy sing new song sits heavens mercy seat holy holy holy lord god almighty come creation sing praise king kings everything adore clothed rainbows living color flashes lightning rolls thunder blessing honor strength glory power wise king holy holy holy lord god almighty come creation sing praise king kings everything adore adore filled wonder awestruck wonder mention name jesus name power breath living water moment mystery oh youre worthy mystery worthy yes holy holy holy lord god almighty come creation sing praise king kings everything adore adore
	Christian

ill bet youve never heard ole marshall dillon say miss kitty ever thought running away settling would marry ask twice begged pretty please lord shed said yes new york minute never tied knot heart wasnt stole kiss rode away never hung brim kittys place shouldve cowboy shouldve learned rope ride wearing sixshooter riding pony cattle drive steal

In [51]:
columns.genre.value_counts()

Christian    94
Country      87
Name: genre, dtype: int64

### Vektörizasyon

Bir sonraki aşama, kodlanmış verilerin gelecekte model üzerinde kullanılabilmesi için metnin sayısal biçimde temsili olan vektörleştirmedir.

Bazen, tek tek kelimelere ek olarak, öğrenme kalitesini artırmak için 2-3 kelimelik kombinasyonların kullanılması da yararlı olacaktır. Burada yerleşik bir n-gram torba yöntemine sahip olan CountVectorizer vektörleştiricisini kullanıyoruz.

n = 1 - unigram (kelimeler eğitim için ayrı ayrı kullanılır)

n = 2 - bigram (eğitim için kelime çiftleri kullanılır)

n = 3 - trigram (eğitim için üç kelime kullanılır)


In [52]:
from sklearn.feature_extraction.text import CountVectorizer

Vectorizer için ngram_range=(1, 3) ayarlayın, 1'den 3'e kadar n'nin tüm değişkenlerini kullanırız ve hem unigramlara hem de bigramlara ve trigramlara başvururuz:

In [53]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

### Sınıflandırma

Sınıflandırma problemi için Bayes teoreminin katı bağımsızlık varsayımlarıyla uygulanmasına dayanan basit bir olasılıksal sınıflandırıcı olan Naive Bayesian Sınıflandırıcı kullanıyoruz.

In [54]:
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

In [55]:
vectorized_x_test = vectorizer.transform(x_test)
x_test

19     worthy lamb slain holy holy sing new song sits...
170    ill bet youve never heard ole marshall dillon ...
27     oh would kind faith takes climb boat im onto c...
122    im walkin floor cant sleep wink true im hoping...
137    well thought id waitin came home last night yo...
                             ...                        
86     calloused bruised dazed confused spirit left w...
87     everybody falls sometimes got ta find strength...
123    ive got tiger tail plain see wont much get wel...
102    crazy im crazy feeling lonely im crazy crazy f...
51     draw close never let go lay hear say im friend...
Name: lemmatized, Length: 73, dtype: object

In [56]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.88      0.97      0.93        38
     Country       0.97      0.86      0.91        35

    accuracy                           0.92        73
   macro avg       0.92      0.92      0.92        73
weighted avg       0.92      0.92      0.92        73

